In [1]:
import os
import pandas as pd
import polars as pl
import excellaint as ea

test_file = "/Users/ct6g18/Python/ExcellAint/test/test_data/2_digit_yr.xlsx"

df = pd.read_excel(test_file)

df["pd_read_dates"] = pd.to_datetime(df["mangled_dates"])

df.iloc[280:295]

/var/folders/r7/qpq_q3zj64nbvr29pvrhky6m0000gq/T/ipykernel_53395/3812195674.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["pd_read_dates"] = pd.to_datetime(df["mangled_dates"])


,mangled_dates,row_id,pd_read_dates
280,12/01/20 04:00,280,2020-12-01 04:00:00
281,12/01/20 05:00,281,2020-12-01 05:00:00
282,12/01/20 06:00,282,2020-12-01 06:00:00
283,12/01/20 07:00,283,2020-12-01 07:00:00
284,12/01/20 08:00,284,2020-12-01 08:00:00
285,12/01/20 09:00,285,2020-12-01 09:00:00
286,12/01/20 10:00,286,2020-12-01 10:00:00
287,12/01/20 11:00,287,2020-12-01 11:00:00
288,13/01/2020 0:00,288,2020-01-13 00:00:00
289,13/01/2020 1:00,289,2020-01-13 01:00:00


It seems to me like the best way to tackle this problem is to first split the string using the datetime_separator. This then gives us separate date and time columns. 

I'm going to ignore anything subsecond for now, because I'd rather focus on the dates, rather than the times - and I think that the dates are more likely to be the source of the error.

In [2]:
print(ea.config)

ExcellAint Configuration:
{   'allow_dayfirst': True,
    'allow_monthfirst': False,
    'allow_yearfirst': True,
    'date_sep': '-',
    'datetime_sep': ' ',
    'day': True,
    'hour': True,
    'minute': True,
    'mode': 'date',
    'month': True,
    'second': True,
    'time_sep': ':',
    'year': True}


In [20]:
df = (
    df
    .with_columns(
        pl.col("mangled_dates").str.split(ea.config.datetime_sep)
                                   .alias("[date,time]"))
    .explode("[date,time]")
    .with_columns(
        ("string_" + pl.arange(0, pl.len()).cast(pl.Utf8).str.zfill(2))
        .over("row_id")
        .alias("col_nm")
    )
    .pivot(
        index=['row_id', 'mangled_dates'],
        values='[date,time]',
        columns='col_nm',
    )
    .rename(
        {
            "string_00": "date_str",
            "string_01": "time_str"
        }
    )
)

df

row_id,mangled_dates,date,time
i64,str,str,str
0,"""01/01/20 12:00…","""01/01/20""","""12:00"""
1,"""01/01/20 01:00…","""01/01/20""","""01:00"""
2,"""01/01/20 02:00…","""01/01/20""","""02:00"""
3,"""01/01/20 03:00…","""01/01/20""","""03:00"""
4,"""01/01/20 04:00…","""01/01/20""","""04:00"""
…,…,…,…
17539,"""31/12/2021 19:…","""31/12/2021""","""19:00"""
17540,"""31/12/2021 20:…","""31/12/2021""","""20:00"""
17541,"""31/12/2021 21:…","""31/12/2021""","""21:00"""
